# Production-Ready Decorators

Reusable decorators for real applications. Copy these into your projects.

## 1. Timer Decorator

In [ ]:
import functools
import time
import logging

def timer(func=None, *, logger=None):
    """
    Log execution time of a function.
    
    Usage:
        @timer
        def my_func(): ...
        
        @timer(logger=my_logger)
        def my_func(): ...
    """
    def decorator(fn):
        @functools.wraps(fn)
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            try:
                return fn(*args, **kwargs)
            finally:
                elapsed = time.perf_counter() - start
                msg = f"{fn.__name__} took {elapsed:.4f}s"
                if logger:
                    logger.info(msg)
                else:
                    print(msg)
        return wrapper
    
    if func is not None:
        return decorator(func)
    return decorator


# Test
@timer
def slow_function():
    time.sleep(0.5)
    return "Done"

result = slow_function()

## 2. Retry Decorator

In [ ]:
import functools
import time
import logging
from typing import Type, Tuple

def retry(
    max_attempts: int = 3,
    delay: float = 1.0,
    backoff: float = 2.0,
    exceptions: Tuple[Type[Exception], ...] = (Exception,),
    logger: logging.Logger | None = None
):
    """
    Retry a function on failure with exponential backoff.
    
    Args:
        max_attempts: Maximum retry attempts
        delay: Initial delay between retries (seconds)
        backoff: Multiplier for delay on each retry
        exceptions: Tuple of exceptions to catch
        logger: Optional logger for retry messages
    
    Usage:
        @retry(max_attempts=3, delay=1.0)
        def call_api(): ...
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            current_delay = delay
            last_exception = None
            
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    last_exception = e
                    if attempt == max_attempts:
                        break
                    
                    msg = f"{func.__name__} attempt {attempt} failed: {e}. Retrying in {current_delay:.1f}s..."
                    if logger:
                        logger.warning(msg)
                    else:
                        print(msg)
                    
                    time.sleep(current_delay)
                    current_delay *= backoff
            
            raise last_exception
        return wrapper
    return decorator


# Test
attempt_count = 0

@retry(max_attempts=3, delay=0.1)
def flaky_function():
    global attempt_count
    attempt_count += 1
    if attempt_count < 3:
        raise ConnectionError("API unavailable")
    return "Success!"

print(flaky_function())

## 3. Log Calls Decorator

In [ ]:
import functools
import logging

def log_calls(logger: logging.Logger | None = None, level: int = logging.INFO):
    """
    Log function calls with arguments and return values.
    
    Args:
        logger: Logger instance (uses print if None)
        level: Logging level
    
    Usage:
        @log_calls(logger=my_logger)
        def my_function(x, y): ...
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Format arguments
            args_repr = [repr(a) for a in args]
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
            signature = ", ".join(args_repr + kwargs_repr)
            
            call_msg = f"Calling {func.__name__}({signature})"
            
            try:
                result = func(*args, **kwargs)
                result_msg = f"{func.__name__} returned {result!r}"
                
                if logger:
                    logger.log(level, call_msg)
                    logger.log(level, result_msg)
                else:
                    print(call_msg)
                    print(result_msg)
                
                return result
            except Exception as e:
                error_msg = f"{func.__name__} raised {type(e).__name__}: {e}"
                if logger:
                    logger.exception(error_msg)
                else:
                    print(error_msg)
                raise
        return wrapper
    return decorator


# Test
@log_calls()
def add(a, b):
    return a + b

add(3, 5)

## 4. Validate Arguments Decorator

In [ ]:
import functools
from typing import Any, Callable, Dict, Type

def validate_args(**validators: Callable[[Any], bool]):
    """
    Validate function arguments using custom validators.
    
    Args:
        **validators: Mapping of arg_name -> validator_function
            Validator should return True if valid, False otherwise
    
    Usage:
        @validate_args(
            age=lambda x: x > 0,
            email=lambda x: '@' in x
        )
        def create_user(name, age, email): ...
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Get function parameter names
            import inspect
            sig = inspect.signature(func)
            bound = sig.bind(*args, **kwargs)
            bound.apply_defaults()
            
            # Validate each argument
            for param_name, validator in validators.items():
                if param_name in bound.arguments:
                    value = bound.arguments[param_name]
                    if not validator(value):
                        raise ValueError(
                            f"Invalid value for '{param_name}': {value!r}"
                        )
            
            return func(*args, **kwargs)
        return wrapper
    return decorator


# Test
@validate_args(
    age=lambda x: isinstance(x, int) and x > 0,
    email=lambda x: isinstance(x, str) and '@' in x
)
def create_user(name: str, age: int, email: str):
    return {"name": name, "age": age, "email": email}

print(create_user("Alice", 30, "alice@example.com"))

try:
    create_user("Bob", -5, "invalid")
except ValueError as e:
    print(f"Validation error: {e}")

## 5. Rate Limiter Decorator

In [ ]:
import functools
import time
from collections import deque

def rate_limit(max_calls: int, period: float):
    """
    Limit function calls to max_calls per period seconds.
    
    Args:
        max_calls: Maximum number of calls allowed
        period: Time window in seconds
    
    Usage:
        @rate_limit(max_calls=10, period=60)  # 10 calls per minute
        def call_api(): ...
    """
    def decorator(func):
        calls = deque()
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            
            # Remove old calls outside the window
            while calls and calls[0] < now - period:
                calls.popleft()
            
            if len(calls) >= max_calls:
                sleep_time = calls[0] - (now - period)
                print(f"Rate limit reached. Waiting {sleep_time:.2f}s...")
                time.sleep(sleep_time)
            
            calls.append(time.time())
            return func(*args, **kwargs)
        return wrapper
    return decorator


# Test
@rate_limit(max_calls=3, period=2)
def api_call(n):
    print(f"Call {n} at {time.time():.2f}")
    return n

for i in range(5):
    api_call(i)

## 6. Memoize with TTL

In [ ]:
import functools
import time
from typing import Any, Dict, Tuple

def memoize_with_ttl(ttl_seconds: float = 300):
    """
    Cache function results with time-to-live expiration.
    
    Args:
        ttl_seconds: How long to keep cached values (default 5 min)
    
    Usage:
        @memoize_with_ttl(ttl_seconds=60)  # Cache for 1 minute
        def fetch_data(key): ...
    """
    def decorator(func):
        cache: Dict[Tuple, Tuple[float, Any]] = {}
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Create cache key from args
            key = (args, tuple(sorted(kwargs.items())))
            now = time.time()
            
            # Check if cached and not expired
            if key in cache:
                timestamp, result = cache[key]
                if now - timestamp < ttl_seconds:
                    return result
            
            # Compute and cache
            result = func(*args, **kwargs)
            cache[key] = (now, result)
            return result
        
        wrapper.cache_clear = lambda: cache.clear()
        wrapper.cache_info = lambda: {"size": len(cache), "ttl": ttl_seconds}
        return wrapper
    return decorator


# Test
@memoize_with_ttl(ttl_seconds=2)
def expensive_computation(x):
    print(f"Computing for {x}...")
    return x ** 2

print(expensive_computation(5))  # Computes
print(expensive_computation(5))  # Cached
time.sleep(2.5)
print(expensive_computation(5))  # Expired, recomputes

## All-in-One Module

Copy this cell to create a `decorators.py` module:

In [ ]:
# decorators.py - Copy this entire cell to a file
"""
Production-ready decorators for Python applications.

Usage:
    from decorators import timer, retry, log_calls, rate_limit
"""

__all__ = ["timer", "retry", "log_calls", "validate_args", "rate_limit", "memoize_with_ttl"]

# Copy the decorator functions from above cells here